In [1]:
# [Note] http://numba.pydata.org/
# [Note] http://numba.pydata.org/numba-doc/0.35.0/index.html
# [Note] conda update numba


import numba
from numba import jit, float32, int32, void, cuda
from numpy import arange
from timeit import default_timer as timer

print("Numba Version", numba.__version__)

# jit decorator tells Numba to compile this function.
# The argument types will be inferred by Numba when function is called.
@jit
def jit_sum_1(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result


@jit(float32(int32[:]))
def jit_sum_2(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result


def sum(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result


# @cuda.jit(void(int32[:]))
# def cuda_jit_sum(arr):
#     M, N = arr.shape
#     result = 0.0
#     for i in range(M):
#         for j in range(N):
#             result += arr[i,j]
#     return result


a = arange(900000000).reshape(30000, 30000)
print(a)

print()

s = timer()
result = jit_sum_1(a)
e = timer()
print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
print(result)

print()

s = timer()
result = jit_sum_2(a)
e = timer()
print("JIT_SUM_2: {:7.6f} ms".format((e - s) * 1000))
print(result)

print()

s = timer()
result = sum(a)
e = timer()
print("NORMAL_SUM: {:7.6f} ms".format((e - s) * 1000))
print(result)

print()

# s = timer()
# result = cuda_jit_sum(a)
# e = timer()
# print("{:7.6f} ms".format((e - s) * 1000))
# print(result)

Numba Version 0.35.0
[[        0         1         2 ...,     29997     29998     29999]
 [    30000     30001     30002 ...,     59997     59998     59999]
 [    60000     60001     60002 ...,     89997     89998     89999]
 ..., 
 [899910000 899910001 899910002 ..., 899939997 899939998 899939999]
 [899940000 899940001 899940002 ..., 899969997 899969998 899969999]
 [899970000 899970001 899970002 ..., 899999997 899999998 899999999]]

JIT_SUM_1: 1460.018000 ms
4.04999999167109e+17

JIT_SUM_2: 1095.918468 ms
4.04999999167109e+17



KeyboardInterrupt: 

### 실험1
- def위에 jit을 적용할 경우와 안 할 경우 얼마만큼의 시간 차이가 발생할까?

In [ ]:
def jit_sum_1(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

@jit
def jit_sum_1_(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

In [6]:
a = arange(10000).reshape(100, 100)
print(a)

print()

s = timer()
result = jit_sum_1(a)
e = timer()
print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
print(result)

print()

s = timer()
result = jit_sum_1_(a)
e = timer()
print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
print(result)

print()

[[   0    1    2 ...,   97   98   99]
 [ 100  101  102 ...,  197  198  199]
 [ 200  201  202 ...,  297  298  299]
 ..., 
 [9700 9701 9702 ..., 9797 9798 9799]
 [9800 9801 9802 ..., 9897 9898 9899]
 [9900 9901 9902 ..., 9997 9998 9999]]

JIT_SUM_1: 2.911480 ms
49995000.0

JIT_SUM_1: 75.974958 ms
49995000.0



- 실험결과
  - 오히려 jit을 적용하지 않은 jit_sum_1() 메소드가 더 빠르다.
  - 아마 원소의 개수를 더 줘야 할 것 같다.

### 실험1 sub-1
- 원소의 개수를 1만에서 1백만으로 100배 늘린다면, 수행 시간이 감소할 것이다.

In [7]:
a = arange(1000000).reshape(1000, 1000)
print(a)

print()

s = timer()
result = jit_sum_1(a)
e = timer()
print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
print(result)

print()

s = timer()
result = jit_sum_1_(a)
e = timer()
print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
print(result)

print()

[[     0      1      2 ...,    997    998    999]
 [  1000   1001   1002 ...,   1997   1998   1999]
 [  2000   2001   2002 ...,   2997   2998   2999]
 ..., 
 [997000 997001 997002 ..., 997997 997998 997999]
 [998000 998001 998002 ..., 998997 998998 998999]
 [999000 999001 999002 ..., 999997 999998 999999]]

JIT_SUM_1: 297.080269 ms
499999500000.0

JIT_SUM_1: 1.128143 ms
499999500000.0



- 실험결과
  - 예상대로 감소했다.
  - 여기서 얻을 수 있는 교훈은, jit을 사용하려면 원소의 개수를 고려해야 한다는 점이다. 원소 개수가 별로 없는 연산의 경우엔 오히려 jit을 쓰는 것이 불이익이다.
- 그럼, 어느 정도의 개수여야 할까?

### 실험1 sub-2
- 원소의 개수를 1만, 4만, 25만, 10만, 50만, 100만으로 각각 실험해본다.

In [20]:
a = {}
a[1] = arange(10000).reshape(100, 100)
a[2] = arange(40000).reshape(200, 200)
a[3] = arange(100000).reshape(100, 1000)
a[4] = arange(250000).reshape(500, 500)
a[5] = arange(500000).reshape(500, 1000)
a[6] = arange(500000).reshape(100, 5000)
a[7] = arange(1000000).reshape(1000, 1000)

print("JIT을 사용하지 않은 케이스")
for key in a:
    s = timer()
    result = jit_sum_1(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))

print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_1_(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))

JIT을 사용하지 않은 케이스
JIT_SUM_1: 2.733223 ms
JIT_SUM_1: 12.139406 ms
JIT_SUM_1: 29.484343 ms
JIT_SUM_1: 70.425865 ms
JIT_SUM_1: 169.344397 ms
JIT_SUM_1: 140.061413 ms
JIT_SUM_1: 300.476829 ms

JIT을 사용한 케이스
JIT_SUM_1: 0.015630 ms
JIT_SUM_1: 0.038769 ms
JIT_SUM_1: 0.094669 ms
JIT_SUM_1: 0.234103 ms
JIT_SUM_1: 0.472197 ms
JIT_SUM_1: 0.475374 ms
JIT_SUM_1: 0.951363 ms


- 실험결과
  - 아까 내린 가정이 흔들린다. 모든 경우에 있어서 jit을 사용한 케이스가 월등히 더 좋다.
  - jit은 '처음에 연산할 때는 오래걸릴 수 있지만, 그 다음 실행부터는 빠르다'라는 문구를 본적 있는 것 같는데, 이것과 관련있는 건가?
  - 그리고 jit은 1만~100만개까지 연산시간의 차이는 무의미한 것으로 보인다.(반면, ndarray만을 사용할 경우, 비례한다.)

### 실험1 sub-3
- 좀 더 늘려보자. 0을 두 개 씩 더 붙인다.

In [21]:
a = {}
a[1] = arange(1000000).reshape(1000, 1000)
a[2] = arange(4000000).reshape(2000, 2000)
a[3] = arange(10000000).reshape(1000, 10000)
a[4] = arange(25000000).reshape(5000, 5000)
a[5] = arange(50000000).reshape(5000, 10000)
a[6] = arange(50000000).reshape(1000, 50000)
a[7] = arange(100000000).reshape(10000, 10000)

print("JIT을 사용하지 않은 케이스")
for key in a:
    s = timer()
    result = jit_sum_1(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))

print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_1_(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))

JIT을 사용하지 않은 케이스
JIT_SUM_1: 301.012691 ms
JIT_SUM_1: 1177.372022 ms
JIT_SUM_1: 2962.175087 ms
JIT_SUM_1: 7227.778586 ms
JIT_SUM_1: 14426.096679 ms
JIT_SUM_1: 14333.019120 ms
JIT_SUM_1: 28877.913244 ms

JIT을 사용한 케이스
JIT_SUM_1: 0.990140 ms
JIT_SUM_1: 3.999558 ms
JIT_SUM_1: 10.187999 ms
JIT_SUM_1: 26.239511 ms
JIT_SUM_1: 56.194355 ms
JIT_SUM_1: 52.216039 ms
JIT_SUM_1: 103.133880 ms


- 실험결과
  - 원소의 개수가 늘어날 수록 증가하는 경향을 띄기는 한다.

### 실험1 sub-4
- 이미 한 번 수행한 연산이라면, 재실행했을 때 시간이 감소할 것인가?

In [22]:
a = {}
a[1] = arange(1000000).reshape(1000, 1000)
a[2] = arange(4000000).reshape(2000, 2000)
a[3] = arange(10000000).reshape(1000, 10000)
a[4] = arange(25000000).reshape(5000, 5000)
a[5] = arange(50000000).reshape(5000, 10000)
a[6] = arange(50000000).reshape(1000, 50000)
a[7] = arange(100000000).reshape(10000, 10000)

print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_1_(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
    
    
print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_1_(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))


JIT을 사용한 케이스
JIT_SUM_1: 0.979696 ms
JIT_SUM_1: 4.119056 ms
JIT_SUM_1: 10.821759 ms
JIT_SUM_1: 26.055557 ms
JIT_SUM_1: 54.098688 ms
JIT_SUM_1: 52.870398 ms
JIT_SUM_1: 103.686453 ms

JIT을 사용한 케이스
JIT_SUM_1: 0.959559 ms
JIT_SUM_1: 4.335598 ms
JIT_SUM_1: 11.890473 ms
JIT_SUM_1: 29.537422 ms
JIT_SUM_1: 51.123961 ms
JIT_SUM_1: 50.447552 ms
JIT_SUM_1: 94.914427 ms


- 실험결과
  - 상관 없다.
  - 도대체 아까 실험 결과는 왜..

In [23]:
a = arange(10000).reshape(100, 100)
print(a)

print()

s = timer()
result = jit_sum_1(a)
e = timer()
print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
print(result)

print()

s = timer()
result = jit_sum_1_(a)
e = timer()
print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
print(result)

print()

[[   0    1    2 ...,   97   98   99]
 [ 100  101  102 ...,  197  198  199]
 [ 200  201  202 ...,  297  298  299]
 ..., 
 [9700 9701 9702 ..., 9797 9798 9799]
 [9800 9801 9802 ..., 9897 9898 9899]
 [9900 9901 9902 ..., 9997 9998 9999]]

JIT_SUM_1: 3.055567 ms
49995000.0

JIT_SUM_1: 0.055956 ms
49995000.0



아까의 실험 코드를 그대로 가져왔는데, 결과가 완전히 다르다. 아까는 왜 그렇게 뜬거였지?

### 실험2
- for문에 붙이는 것과 def에 붙이는 것에 차이가 있나?

In [25]:
a = {}
a[1] = arange(1000000).reshape(1000, 1000)
a[2] = arange(4000000).reshape(2000, 2000)
a[3] = arange(10000000).reshape(1000, 10000)
a[4] = arange(25000000).reshape(5000, 5000)
a[5] = arange(50000000).reshape(5000, 10000)
a[6] = arange(50000000).reshape(1000, 50000)
a[7] = arange(100000000).reshape(10000, 10000)

def jit_sum_2(arr):
    M, N = arr.shape
    result = 0.0
    @jit(float32(int32[:]))
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

@jit(float32(int32[:]))
def jit_sum_2_(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result


print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_2(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
    
    
print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_2_(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))



SyntaxError: invalid syntax (<ipython-input-25-641b411a9c33>, line 14)

- 실험결과
  - jit은 for문에는 사용이 불가능하다. def문에만 사용 가능한 것 같다.

음? 지금보니까, jit에 parameter로 float32(int32[:]) 이런걸 주는 게 관찰된다. jit은 함수인가? 이 parameter는 뭐지?
  - jit은 함수일 것이다. parameter를 받아들이니까.
    - 그렇다면 왜 그냥 'jit'만을 쳐도 가능한거지? 함수 호출이라면 반드시 func()와 같이 괄호가 붙어야 하는데.

In [26]:
a = {}
a[1] = arange(1000000).reshape(1000, 1000)
a[2] = arange(4000000).reshape(2000, 2000)
a[3] = arange(10000000).reshape(1000, 10000)
a[4] = arange(25000000).reshape(5000, 5000)
a[5] = arange(50000000).reshape(5000, 10000)
a[6] = arange(50000000).reshape(1000, 50000)
a[7] = arange(100000000).reshape(10000, 10000)

@jit(float32(int32[:]))
def jit_sum_2_(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result
    
print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_2_(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))


JIT을 사용한 케이스
JIT_SUM_1: 70.582567 ms
JIT_SUM_1: 3.854764 ms
JIT_SUM_1: 10.299260 ms
JIT_SUM_1: 28.864715 ms
JIT_SUM_1: 53.326796 ms
JIT_SUM_1: 81.115621 ms
JIT_SUM_1: 160.019110 ms


- 실험결과
  - 처음에 또 높게 나왔다. 혹시 jit 함수(?)를 처음 호출할 때 시간이 많이 걸리는건가? 매번 첫 호출 때 시간이 높게 나타났다. (그 매번이 두 번이지만..)
- float32(int32[:])의 의미는 혹시.. float 타입인 변수 result, 그리고 result에 연산되어 들어가는 int 타입의 변수 arr[i,j]를 의미하는 건가?
- 만약 그렇다면, 실험1번에서의 코드를 다음과 같이 돌릴 수도 있지 않을까?

In [27]:
@jit
def jit_sum_2_1(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

@jit(int32[:])
def jit_sum_2_2(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

@jit(int32(int32[:]))
def jit_sum_2_3(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

@jit(float32(int32[:]))
def jit_sum_2_4(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

a = {}
a[1] = arange(1000000).reshape(1000, 1000)
a[2] = arange(4000000).reshape(2000, 2000)
a[3] = arange(10000000).reshape(1000, 10000)
a[4] = arange(25000000).reshape(5000, 5000)
a[5] = arange(50000000).reshape(5000, 10000)
a[6] = arange(50000000).reshape(1000, 50000)
a[7] = arange(100000000).reshape(10000, 10000)
    
print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_2_1(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
    
print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_2_2(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
    
print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_2_3(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
    
print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_2_4(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))

AttributeError: 'Array' object has no attribute '__defaults__'

In [29]:
@jit
def jit_sum_2_1(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

# @jit(int32[:])
# def jit_sum_2_2(arr):
#     M, N = arr.shape
#     result = 0.0
#     for i in range(M):
#         for j in range(N):
#             result += arr[i,j]
#     return result

@jit(int32(int32[:]))
def jit_sum_2_3(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

@jit(float32(int32[:]))
def jit_sum_2_4(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

a = {}
a[1] = arange(1000000).reshape(1000, 1000)
a[2] = arange(4000000).reshape(2000, 2000)
a[3] = arange(10000000).reshape(1000, 10000)
a[4] = arange(25000000).reshape(5000, 5000)
a[5] = arange(50000000).reshape(5000, 10000)
a[6] = arange(50000000).reshape(1000, 50000)
a[7] = arange(100000000).reshape(10000, 10000)
    
print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_2_1(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
    
# print()
# print("JIT을 사용한 케이스")
# for key in a:
#     s = timer()
#     result = jit_sum_2_2(a[key])
#     e = timer()
#     print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
    
print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_2_3(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))
    
print()
print("JIT을 사용한 케이스")
for key in a:
    s = timer()
    result = jit_sum_2_4(a[key])
    e = timer()
    print("JIT_SUM_1: {:7.6f} ms".format((e - s) * 1000))


JIT을 사용한 케이스
JIT_SUM_1: 73.270995 ms
JIT_SUM_1: 4.121462 ms
JIT_SUM_1: 9.928277 ms
JIT_SUM_1: 25.427817 ms
JIT_SUM_1: 52.508160 ms
JIT_SUM_1: 50.112708 ms
JIT_SUM_1: 97.521457 ms

JIT을 사용한 케이스
JIT_SUM_1: 77.781569 ms
JIT_SUM_1: 4.084083 ms
JIT_SUM_1: 10.767192 ms
JIT_SUM_1: 25.671278 ms
JIT_SUM_1: 49.007850 ms
JIT_SUM_1: 49.388598 ms
JIT_SUM_1: 104.422877 ms

JIT을 사용한 케이스
JIT_SUM_1: 93.934923 ms
JIT_SUM_1: 4.268976 ms
JIT_SUM_1: 9.858732 ms
JIT_SUM_1: 24.172018 ms
JIT_SUM_1: 50.802596 ms
JIT_SUM_1: 52.201114 ms
JIT_SUM_1: 101.079778 ms


- 해석
  - @jit(int32[:]) 이 부분이 안되는 이유는 아마 result의 타입 자체가 float이라서 그럴 것이다.
    - 그런데 그렇다면 @jit(int32(int32[:])) 얘는 왜 되는거지?
  - jit에 특정 인자를 준다고 해서 시간이 줄어드는 것처럼 보이지는 않는다.
    - 그럼 도대체 jit에 parameter는 왜 주는건가? 안주고 그냥 jit만 붙여도 되는 것 아닌가?